In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/ransomware/'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-02 17:32:17.262052: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 101 images belonging to 10 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(101, 10)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.2)

In [5]:
X_train.shape

(80, 64, 64, 3)

In [6]:
X_test.shape

(21, 64, 64, 3)

In [7]:
y_train.shape

(80, 10)

In [8]:
y_test.shape

(21, 10)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 10

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(31, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(17, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))
    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-02 17:32:19.132689: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-02 17:32:19.402141: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-02 17:32:19.402160: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-02 17:32:19.402415: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 17:32:19.410391: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-02 17:32:19.4111

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 31)        868       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 31)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 17)        4760      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 17)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 17)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3332)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(80, 10)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([4, 9, 8, 0, 0, 1, 8, 5, 9, 3, 2, 9, 8, 3, 3, 7, 1, 7, 1, 5, 2, 1,
       0, 1, 3, 9, 6, 3, 7, 5, 4, 2, 6, 8, 6, 7, 3, 9, 8, 7, 2, 7, 5, 8,
       9, 6, 4, 7, 8, 3, 9, 4, 5, 0, 0, 5, 5, 1, 0, 4, 4, 2, 8, 2, 0, 3,
       9, 6, 5, 3, 7, 7, 4, 8, 6, 3, 4, 0, 7, 1])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)

2023-05-02 17:32:19.602038: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 2s 313ms/step - loss: 0.7175 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4878 - val_loss: 0.5816 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.4544
Epoch 2/100
3/3 [==============================] - 0s 54ms/step - loss: 0.5862 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4881 - val_loss: 0.4668 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.4429
Epoch 3/100
3/3 [==============================] - 0s 65ms/step - loss: 0.4743 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.5180 - val_loss: 0.4024 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.4764
Epoch 4/100
3/3 [==============================] - 0s 63ms/step - loss: 0.4539 - f1_m: 0.0542 - precision_m: 0.1354 - recall_m: 0.0339 - auc: 0.5064 - val_loss: 0.35

3/3 [==============================] - 0s 51ms/step - loss: 0.2055 - f1_m: 0.5379 - precision_m: 0.7964 - recall_m: 0.4062 - auc: 0.9170 - val_loss: 0.1716 - val_f1_m: 0.6000 - val_precision_m: 1.0000 - val_recall_m: 0.4286 - val_auc: 0.9690
Epoch 32/100
3/3 [==============================] - 0s 63ms/step - loss: 0.1834 - f1_m: 0.5819 - precision_m: 0.8065 - recall_m: 0.4570 - auc: 0.9552 - val_loss: 0.1605 - val_f1_m: 0.6452 - val_precision_m: 1.0000 - val_recall_m: 0.4762 - val_auc: 0.9762
Epoch 33/100
3/3 [==============================] - 0s 58ms/step - loss: 0.1795 - f1_m: 0.6222 - precision_m: 0.8471 - recall_m: 0.4948 - auc: 0.9531 - val_loss: 0.1560 - val_f1_m: 0.7273 - val_precision_m: 1.0000 - val_recall_m: 0.5714 - val_auc: 0.9763
Epoch 34/100
3/3 [==============================] - 0s 66ms/step - loss: 0.1763 - f1_m: 0.6305 - precision_m: 0.8282 - recall_m: 0.5104 - auc: 0.9484 - val_loss: 0.1578 - val_f1_m: 0.6667 - val_precision_m: 0.9167 - val_recall_m: 0.5238 - val_auc: 

3/3 [==============================] - 0s 58ms/step - loss: 0.0396 - f1_m: 0.9717 - precision_m: 1.0000 - recall_m: 0.9453 - auc: 0.9993 - val_loss: 0.1738 - val_f1_m: 0.7500 - val_precision_m: 0.7895 - val_recall_m: 0.7143 - val_auc: 0.9458
Epoch 64/100
3/3 [==============================] - 0s 63ms/step - loss: 0.0332 - f1_m: 0.9909 - precision_m: 0.9909 - recall_m: 0.9909 - auc: 0.9999 - val_loss: 0.1884 - val_f1_m: 0.7368 - val_precision_m: 0.8235 - val_recall_m: 0.6667 - val_auc: 0.9418
Epoch 65/100
3/3 [==============================] - 0s 67ms/step - loss: 0.0379 - f1_m: 0.9726 - precision_m: 0.9903 - recall_m: 0.9557 - auc: 0.9996 - val_loss: 0.1905 - val_f1_m: 0.6829 - val_precision_m: 0.7000 - val_recall_m: 0.6667 - val_auc: 0.9418
Epoch 66/100
3/3 [==============================] - 0s 63ms/step - loss: 0.0341 - f1_m: 0.9850 - precision_m: 0.9896 - recall_m: 0.9805 - auc: 0.9999 - val_loss: 0.1774 - val_f1_m: 0.6829 - val_precision_m: 0.7000 - val_recall_m: 0.6667 - val_auc: 

3/3 [==============================] - 0s 60ms/step - loss: 0.0133 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - auc: 1.0000 - val_loss: 0.2019 - val_f1_m: 0.7619 - val_precision_m: 0.7619 - val_recall_m: 0.7619 - val_auc: 0.9327
Epoch 96/100
3/3 [==============================] - 0s 57ms/step - loss: 0.0152 - f1_m: 0.9896 - precision_m: 0.9896 - recall_m: 0.9896 - auc: 1.0000 - val_loss: 0.1814 - val_f1_m: 0.7143 - val_precision_m: 0.7143 - val_recall_m: 0.7143 - val_auc: 0.9505
Epoch 97/100
3/3 [==============================] - 0s 56ms/step - loss: 0.0214 - f1_m: 0.9855 - precision_m: 0.9909 - recall_m: 0.9805 - auc: 1.0000 - val_loss: 0.1692 - val_f1_m: 0.7317 - val_precision_m: 0.7500 - val_recall_m: 0.7143 - val_auc: 0.9482
Epoch 98/100
3/3 [==============================] - 0s 59ms/step - loss: 0.0137 - f1_m: 0.9855 - precision_m: 0.9909 - recall_m: 0.9805 - auc: 0.9999 - val_loss: 0.1649 - val_f1_m: 0.7619 - val_precision_m: 0.7619 - val_recall_m: 0.7619 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 30ms/step - loss: 0.1920 - f1_m: 0.7143 - precision_m: 0.7143 - recall_m: 0.7143 - auc: 0.9058


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 30ms/step - loss: 0.1920 - f1_m: 0.7143 - precision_m: 0.7143 - recall_m: 0.7143 - auc: 0.9058


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.7142856121063232
